In [7]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val userSdk = UserSdk()
val username = "Johann"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=d70c3c8c-7853-4a91-a282-35c47a69e015, username=Johann)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import kotlinx.coroutines.runBlocking

val fundSdk = FundSdk()
val expenseFund = runBlocking {
    fundSdk.listFunds(user.id).items.find { it.name.value == "Expenses" }
}
expenseFund

FundTO(id=c0170693-48eb-4045-854e-2c4eec60a642, name=Expenses)

In [4]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, ReportViewType.EXPENSE)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=aa9f837b-84fa-4e96-b34d-998341bbbd6a, name=Expense report, fundId=c0170693-48eb-4045-854e-2c4eec60a642, type=EXPENSE)

In [14]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-06-30")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
} as ExpenseReportDataTO
report

ExpenseReportDataTO(viewId=aa9f837b-84fa-4e96-b34d-998341bbbd6a, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-06-30), granularity=MONTHLY), data=[DataItem(timeBucket=2019-01-01, amount=2481.52), DataItem(timeBucket=2019-02-01, amount=2703.56), DataItem(timeBucket=2019-03-01, amount=6116.18), DataItem(timeBucket=2019-04-01, amount=-908.08), DataItem(timeBucket=2019-05-01, amount=-878.38), DataItem(timeBucket=2019-06-01, amount=0.0)])

In [22]:
import java.math.BigDecimal

data class MonthlyExpense(val month: LocalDate, val amount: Double)

val monthlyReportDF = report.data
    .map {
        MonthlyExpense(it.timeBucket, it.amount.toDouble())
    }
    .toDataFrame()

In [48]:
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations

plot(monthlyReportDF) {
    x("month") {

    }
    y("amount")

    line {
        color = Color.hex("#6e5596")
    }
    points { }
    layout {
        title = "Monthly expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="nEEWpb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"amount":[2481.52,2703.56,6116.18,-908.08,-878.38,0.0],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"amount"
},
"stat":"identity",
"color":"#6e5596",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"amount"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("nEEWpb");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 
 
 -1,000 
 
 
 
 
 
 
 -500 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 
 
 Monthly expenses 
 
 
 
 
 amount 
 
 
 
 
 month